In [33]:
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv('House_dataset/1976-2020-house.csv')

In [42]:
df.shape

(31101, 20)

In [43]:
# drop unused columns and nulls
df = df.drop(columns=['mode', 'version', 'office', 'fusion_ticket', 'unofficial', 'writein',
                      'special', 'runoff', 'stage', 'state_cen', 'state_ic'])
df = df.dropna()
# df = df.reset_index()
df.shape

(27472, 9)

In [44]:
# drop counties reporting zero total votes
df = df.loc[df['totalvotes'] != 0]
df.shape

(27471, 9)

In [45]:
# get % of votes each party received in each county
temp = []

for index, row in df.iterrows():
    temp.append(round(row['candidatevotes'] / row['totalvotes'], 4))
df['percentage'] = temp

In [46]:
valid_parties = ['DEMOCRAT', 'REPUBLICAN', 'LIBERTARIAN', 'INDEPENDENT', 'GREEN', 'OTHER']

for index, row in df.iterrows():
    if row['party'] not in valid_parties:
        df.loc[index,'party'] = 'OTHER'
    

In [47]:
counter = {}
for index, row in df.iterrows():
    if row['party'] in counter:
        counter[row['party']] += 1
    else:
        counter[row['party']] = 1
        
print(counter)

{'DEMOCRAT': 9501, 'REPUBLICAN': 9264, 'OTHER': 4349, 'INDEPENDENT': 1166, 'LIBERTARIAN': 2683, 'GREEN': 508}


In [11]:
df.head(10)

,index,year,state,state_po,state_fips,district,candidate,party,candidatevotes,totalvotes,percentage
0,0,1976,ALABAMA,AL,1,1,BILL DAVENPORT,DEMOCRAT,58906,157170,0.3748
1,1,1976,ALABAMA,AL,1,1,JACK EDWARDS,REPUBLICAN,98257,157170,0.6252
2,3,1976,ALABAMA,AL,1,2,J. CAROLE KEAHEY,DEMOCRAT,66288,156362,0.4239
3,5,1976,ALABAMA,AL,1,2,"WILLIAM L. """"BILL"""" DICKINSON",REPUBLICAN,90069,156362,0.5760
4,6,1976,ALABAMA,AL,1,3,BILL NICHOLS,DEMOCRAT,106935,108048,0.9897
5,8,1976,ALABAMA,AL,1,3,OGBURN GARDNER,OTHER,1111,108048,0.0103
6,9,1976,ALABAMA,AL,1,4,TOM BEVILL,DEMOCRAT,141490,176022,0.8038
7,10,1976,ALABAMA,AL,1,4,LEONARD WILSON,REPUBLICAN,34531,176022,0.1962
8,12,1976,ALABAMA,AL,1,5,RONNIE G. FLIPPO,DEMOCRAT,113553,113560,0.9999
9,14,1976,ALABAMA,AL,1,6,BILLY E. DORSEY,OTHER,1021,162518,0.0063


In [48]:
#create new dataframe with desired data config
curr_dist = 'AL1'
new_df = []
temp_dict = {}
for index, row in df.iterrows():
    if row['state_po'] + str(row['district']) == curr_dist:
        temp_dict['year'] = row['year']
        temp_dict['state'] = row['state_po']
        temp_dict['state_fips'] = int(row['state_fips'])
        temp_dict['district'] = row['district']
        temp_dict['total_no'] = row['totalvotes']
        curr_party = row['party'][0].lower()
        
        if curr_party == 'd':
            temp_dict['dem_no'] = row['candidatevotes']
            temp_dict['dem_pct'] = row['percentage']
        elif curr_party == 'r':
            temp_dict['rep_no'] = row['candidatevotes']
            temp_dict['rep_pct'] = row['percentage']
        elif curr_party == 'l':
            temp_dict['lib_no'] = row['candidatevotes']
            temp_dict['lib_pct'] = row['percentage']
        elif curr_party == 'g':
            temp_dict['grn_no'] = row['candidatevotes']
            temp_dict['grn_pct'] = row['percentage']
        else:
            temp_dict['oth_no'] = row['candidatevotes']
            temp_dict['oth_pct'] = row['percentage']
    else:
        copy = temp_dict.copy()
        new_df.append(copy)
        temp_dict.clear()
        
        temp_dict['year'] = row['year']
        temp_dict['state'] = row['state_po']
        temp_dict['state_fips'] = int(row['state_fips'])
        temp_dict['district'] = row['district']
        temp_dict['total_no'] = row['totalvotes']
        curr_party = row['party'][0].lower()

        if curr_party == 'd':
            temp_dict['dem_no'] = row['candidatevotes']
            temp_dict['dem_pct'] = row['percentage']
        elif curr_party == 'r':
            temp_dict['rep_no'] = row['candidatevotes']
            temp_dict['rep_pct'] = row['percentage']
        elif curr_party == 'l':
            temp_dict['lib_no'] = row['candidatevotes']
            temp_dict['lib_pct'] = row['percentage']
        elif curr_party == 'g':
            temp_dict['grn_no'] = row['candidatevotes']
            temp_dict['grn_pct'] = row['percentage']
        else:
            temp_dict['oth_no'] = row['candidatevotes']
            temp_dict['oth_pct'] = row['percentage']
    
    curr_dist = row['state_po'] + str(row['district'])
    

new_df = pd.DataFrame(new_df)
new_df = new_df.set_index(("state_fips","year","district"))
new_df = new_df.drop(columns=[['state_fips','year','district']])
new_df = new_df.replace(np.nan, 0)

KeyError: "None of [('state_fips', 'year', 'district')] are in the columns"

In [38]:
new_df.head(20)

,year,state,state_fips,district,total_no,dem_no,dem_pct,rep_no,rep_pct,oth_no,oth_pct,lib_no,lib_pct,grn_no,grn_pct
0,1976,AL,1,1,157170,58906.0,0.3748,98257.0,0.6252,0.0,0.0000,0.0,0.0000,0.0,0.0
1,1976,AL,1,2,156362,66288.0,0.4239,90069.0,0.5760,0.0,0.0000,0.0,0.0000,0.0,0.0
2,1976,AL,1,3,108048,106935.0,0.9897,0.0,0.0000,1111.0,0.0103,0.0,0.0000,0.0,0.0
3,1976,AL,1,4,176022,141490.0,0.8038,34531.0,0.1962,0.0,0.0000,0.0,0.0000,0.0,0.0
4,1976,AL,1,5,113560,113553.0,0.9999,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0
5,1976,AL,1,6,162518,69384.0,0.4269,92113.0,0.5668,1021.0,0.0063,0.0,0.0000,0.0,0.0
6,1976,AL,1,7,110501,110496.0,1.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0
7,1976,AK,2,0,118208,34194.0,0.2893,83722.0,0.7083,0.0,0.0000,0.0,0.0000,0.0,0.0
8,1976,AZ,4,1,168119,68404.0,0.4069,96397.0,0.5734,1040.0,0.0062,2278.0,0.0135,0.0,0.0
9,1976,AZ,4,2,182128,106054.0,0.5823,71765.0,0.3940,0.0,0.0000,4309.0,0.0237,0.0,0.0


In [36]:
for index, row in new_df.iterrows():
    if row['dem_pct'] + row['rep_pct'] + row['lib_pct'] + row['grn_pct'] + row['oth_pct'] < 0.95:
        print(index, row['year'], row['state'], row['district'])

8 1976 CO 4
34 1976 NJ 15
36 1976 NY 3
36 1976 NY 4
36 1976 NY 7
36 1976 NY 8
36 1976 NY 10
36 1976 NY 13
36 1976 NY 14
36 1976 NY 15
36 1976 NY 16
36 1976 NY 17
36 1976 NY 18
36 1976 NY 19
36 1976 NY 20
36 1976 NY 23
36 1976 NY 24
36 1976 NY 29
36 1976 NY 38
39 1976 OH 18
39 1976 OH 20
41 1976 OR 4
49 1976 UT 2
54 1976 WV 4
23 1978 ME 2
25 1978 MA 9
28 1978 MS 4
34 1978 NJ 9
36 1978 NY 4
36 1978 NY 7
36 1978 NY 8
36 1978 NY 9
36 1978 NY 14
36 1978 NY 15
36 1978 NY 16
36 1978 NY 17
36 1978 NY 19
36 1978 NY 22
36 1978 NY 23
36 1978 NY 24
36 1978 NY 26
36 1978 NY 37
36 1978 NY 38
39 1978 OH 22
42 1978 PA 25
6 1980 CA 11
22 1980 LA 2
22 1980 LA 3
22 1980 LA 4
22 1980 LA 5
22 1980 LA 8
36 1980 NY 1
36 1980 NY 2
36 1980 NY 3
36 1980 NY 4
36 1980 NY 5
36 1980 NY 6
36 1980 NY 7
36 1980 NY 8
36 1980 NY 11
36 1980 NY 13
36 1980 NY 14
36 1980 NY 15
36 1980 NY 16
36 1980 NY 17
36 1980 NY 19
36 1980 NY 20
36 1980 NY 30
36 1980 NY 32
36 1980 NY 33
36 1980 NY 37
36 1980 NY 38
42 1980 PA 1
50 1980 VT